In [43]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
#Set paths 
data_dir = 'D:/mcadata/'  # D: drive

# Define classes 
classes = ['bacteria', 'fungus', 'healthy', 'pests', 'virus']
num_classes = len(classes)

# Parameters
img_size = (150, 150)  # Reduced size for memory efficiency
batch_size = 32
epochs = 15
validation_split = 0.2  # 80-20 split

In [39]:
# data generators with 80-20 split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Training generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    classes=classes
)

Found 4000 images belonging to 5 classes.


In [46]:
# Validation generator
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    classes=classes
)

Found 1000 images belonging to 5 classes.


## Customized CNN

In [47]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),

    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # 5 classes
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [48]:
history = model.fit(train_generator, validation_data=val_generator, epochs=25, callbacks=[early_stop])


Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 44s 341ms/step - accuracy: 0.4786 - loss: 2.9660 - val_accuracy: 0.2390 - val_loss: 6.2519
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 339ms/step - accuracy: 0.6181 - loss: 1.1814 - val_accuracy: 0.2250 - val_loss: 9.2720
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 339ms/step - accuracy: 0.6882 - loss: 0.9056 - val_accuracy: 0.2470 - val_loss: 6.3997
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 339ms/step - accuracy: 0.7086 - loss: 0.7845 - val_accuracy: 0.4070 - val_loss: 3.6672
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 342ms/step - accuracy: 0.7330 - loss: 0.8208 - val_accuracy: 0.4610 - val_loss: 2.6565
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 337ms/step - accuracy: 0.7343 - loss: 0.7711 - val_accuracy: 0.6350 - val_loss: 1.2241
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 44s 348ms/step - accuracy: 0.7619 - loss: 0.6461 - val_accuracy: 0.5130 - val_loss: 1.5767
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 43s 343ms/step - accuracy: 0.7829 - loss: 0

In [54]:
#Evaluate the model
test_loss, test_acc = model.evaluate(val_generator)
print(f"\nTest accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.7444 - loss: 0.9039 

Test accuracy: 0.7350, Test loss: 0.9868


In [49]:
# full model (architecture + weights + optimizer state)
model.save('plant_doctor.h5')


In [50]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('plant_doctor.h5')


In [53]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Define your class labels in the same order
classes = ['bacteria', 'fungus', 'healthy', 'pests', 'virus']

# Load and preprocess the image
img_path = "D:\mcadata\healthy\hea (131).JPG"  # Change to your image path
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Rescale like training data
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
predicted_class = classes[np.argmax(prediction)]

print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: healthy


In [62]:
# Load the model globally so it's only loaded once
def predict_image(img_path):
    if model is None:
        raise Exception("Model failed to load")
    
    # Load and preprocess the image
    target_size = (150, 150)  # Adjust this to match your model's expected input
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Check if model expects normalized inputs (0-1)
    if model.input_shape[-1] == 3:  # If expecting RGB
        img_array = img_array / 255.0
    
    # Verify the shape matches exactly what the model expects
    if img_array.shape[1:] != model.input_shape[1:]:
        raise ValueError(f"Input shape {img_array.shape[1:]} doesn't match model's expected input {model.input_shape[1:]}")
    
    # Make prediction
    prediction = model.predict(img_array)
    
    # Assuming your model has these classes - adjust according to your model
    classes = ['bacterial', 'fungus', 'healthy', 'pest', 'virus']
    
    # Get the predicted class and confidence
    predicted_class = classes[np.argmax(prediction)]
    confidence = np.max(prediction)
    
    return predicted_class, confidence

## tkinter

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import datetime
import webbrowser


class LeafDoctorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Leaf Doctor - Plant Disease Prediction")
        self.root.geometry("900x700")
        self.root.configure(bg="#f0f8ff")
        
        # Set window icon
        try:
            self.root.iconbitmap("leaf_icon.ico")
        except:
            pass
        
        self.create_widgets()
        self.create_remedies_database()
        
    def create_widgets(self):
        # Header Frame
        header_frame = tk.Frame(self.root, bg="#2e8b57")
        header_frame.pack(fill="x", padx=10, pady=10)
        
        # Logo and Title
        try:
            logo_img = Image.open("leaf_logo.png").resize((60, 60))
            self.logo = ImageTk.PhotoImage(logo_img)
            logo_label = tk.Label(header_frame, image=self.logo, bg="#2e8b57")
            logo_label.pack(side="left", padx=10)
        except:
            pass
        
        title_label = tk.Label(header_frame, 
                             text="LEAF DOCTOR", 
                             font=("Helvetica", 24, "bold"), 
                             fg="white", 
                             bg="#2e8b57")
        title_label.pack(side="left", padx=10)
        
        subtitle_label = tk.Label(header_frame, 
                                text="Plant Disease Diagnosis System", 
                                font=("Helvetica", 12), 
                                fg="white", 
                                bg="#2e8b57")
        subtitle_label.pack(side="left", padx=10)
        
        # Main Content Frame
        main_frame = tk.Frame(self.root, bg="#f0f8ff")
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Upload Section
        upload_frame = tk.LabelFrame(main_frame, 
                                    text=" Upload Leaf Image ", 
                                    font=("Helvetica", 12, "bold"),
                                    bg="#e6f3ed",
                                    padx=10,
                                    pady=10)
        upload_frame.pack(fill="x", pady=(0, 20))
        
        # Upload button with icon
        upload_icon = self.create_icon("\u2191", "#4CAF50")  # Up arrow icon
        self.upload_btn = tk.Button(upload_frame, 
                                   image=upload_icon,
                                   compound="left",
                                   text="Browse Image", 
                                   font=("Helvetica", 11),
                                   bg="#4CAF50",
                                   fg="white",
                                   command=self.upload_image)
        self.upload_btn.image = upload_icon
        self.upload_btn.pack(pady=10)
        
        # Image Display
        self.image_frame = tk.Frame(main_frame, 
                                   bg="white", 
                                   width=300, 
                                   height=300,
                                   relief="groove",
                                   borderwidth=2)
        self.image_frame.pack_propagate(False)
        self.image_frame.pack(pady=(0, 20))
        
        self.image_label = tk.Label(self.image_frame, bg="white")
        self.image_label.pack(fill="both", expand=True)
        
        # Prediction Section
        predict_frame = tk.Frame(main_frame, bg="#f0f8ff")
        predict_frame.pack(fill="x")
        
        predict_icon = self.create_icon("\u2695", "#FF5722")  # Medical symbol
        self.predict_btn = tk.Button(predict_frame, 
                                    image=predict_icon,
                                    compound="left",
                                    text="Diagnose Disease", 
                                    font=("Helvetica", 12, "bold"),
                                    bg="#FF5722",
                                    fg="white",
                                    command=self.predict_disease)
        self.predict_btn.image = predict_icon
        self.predict_btn.pack(pady=10)
        
        # Results Section
        results_frame = tk.LabelFrame(main_frame, 
                                     text=" Diagnosis Results ", 
                                     font=("Helvetica", 12, "bold"),
                                     bg="#e6f3ed",
                                     padx=10,
                                     pady=10)
        results_frame.pack(fill="both", expand=True)
        
        # Results grid
        self.create_results_grid(results_frame)
        
        # Footer
        footer_frame = tk.Frame(self.root, bg="#2e8b57")
        footer_frame.pack(fill="x", padx=10, pady=10)
        
        footer_label = tk.Label(footer_frame, 
                               text=f"© {datetime.datetime.now().year} Leaf Doctor - Plant Health Monitoring System", 
                               font=("Helvetica", 9), 
                               fg="white", 
                               bg="#2e8b57")
        footer_label.pack()
        
    def create_results_grid(self, parent):
        # Grid layout for results
        labels = ["Date & Time:", "Disease Detected:", "Confidence Level:", 
                 "Recommended Treatment:", "Prevention Measures:"]
        
        self.result_vars = {}
        
        for i, label_text in enumerate(labels):
            # Label
            label = tk.Label(parent, 
                            text=label_text, 
                            font=("Helvetica", 10, "bold"), 
                            bg="#e6f3ed",
                            anchor="w")
            label.grid(row=i, column=0, sticky="w", padx=5, pady=5)
            
            # Value display
            value_var = tk.StringVar()
            value_var.set("Not available")
            
            if label_text in ["Recommended Treatment:", "Prevention Measures:"]:
                # Use text widget for multi-line content
                text_widget = tk.Text(parent, 
                                     height=4 if label_text == "Recommended Treatment:" else 5,
                                     width=60,
                                     font=("Helvetica", 9),
                                     wrap="word",
                                     bg="white",
                                     relief="flat")
                text_widget.grid(row=i, column=1, sticky="w", padx=5, pady=5)
                text_widget.insert("1.0", "No diagnosis performed yet")
                text_widget.config(state="disabled")
                self.result_vars[label_text] = text_widget
            else:
                # Use label for single-line content
                value_label = tk.Label(parent, 
                                      textvariable=value_var, 
                                      font=("Helvetica", 9), 
                                      bg="white",
                                      relief="sunken",
                                      padx=5,
                                      pady=2,
                                      width=60,
                                      anchor="w")
                value_label.grid(row=i, column=1, sticky="w", padx=5, pady=5)
                self.result_vars[label_text] = value_var
        
        # Add a learn more button
        self.learn_more_btn = tk.Button(parent,
                                       text="Learn More Online",
                                       font=("Helvetica", 10),
                                       bg="#2196F3",
                                       fg="white",
                                       state="disabled",
                                       command=self.open_web_resources)
        self.learn_more_btn.grid(row=len(labels), column=1, sticky="e", pady=10)
    
    def create_icon(self, symbol, color):
        # Create a simple icon using unicode symbols
        icon_img = Image.new('RGB', (20, 20), color)
        return ImageTk.PhotoImage(icon_img)
    
    def upload_image(self):
        file_path = filedialog.askopenfilename(
            title="Select Leaf Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        
        if file_path:
            self.current_image_path = file_path
            try:
                img = Image.open(file_path)
                img.thumbnail((300, 300))
                photo = ImageTk.PhotoImage(img)
                
                self.image_label.configure(image=photo)
                self.image_label.image = photo
                
                # Reset previous results
                self.clear_results()
                
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")
    
    def clear_results(self):
        for key, widget in self.result_vars.items():
            if isinstance(widget, tk.Text):
                widget.config(state="normal")
                widget.delete("1.0", "end")
                widget.insert("1.0", "No diagnosis performed yet")
                widget.config(state="disabled")
            else:
                widget.set("Not available")
        
        self.learn_more_btn.config(state="disabled")
    
    def predict_disease(self):
        if not hasattr(self, 'current_image_path'):
            messagebox.showwarning("Warning", "Please upload an image first!")
            return
        
        try:
            # Get prediction from your model
            disease, confidence = predict_image(self.current_image_path)
            
            # Get current datetime
            now = datetime.datetime.now()
            dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
            
            # Update results
            self.result_vars["Date & Time:"].set(dt_string)
            self.result_vars["Disease Detected:"].set(disease)
            self.result_vars["Confidence Level:"].set(f"{confidence:.2%}")
            
            # Get remedies and prevention
            remedies = self.get_remedies(disease)
            
            # Update text widgets
            for widget in self.result_vars.values():
                if isinstance(widget, tk.Text):
                    widget.config(state="normal")
                    widget.delete("1.0", "end")
            
            self.result_vars["Recommended Treatment:"].insert("1.0", remedies["treatment"])
            self.result_vars["Prevention Measures:"].insert("1.0", remedies["prevention"])
            
            for widget in self.result_vars.values():
                if isinstance(widget, tk.Text):
                    widget.config(state="disabled")
            
            # Enable learn more button
            self.learn_more_btn.config(state="normal")
            
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {str(e)}")
    
    def create_remedies_database(self):
        # Database of plant disease remedies and prevention measures
        self.remedies_db = {
            "bacteria": {
                "treatment": "1. Remove and destroy infected leaves\n2. Apply copper-based bactericides\n3. Use streptomycin spray for severe cases\n4. Avoid overhead watering",
                "prevention": "1. Use disease-free seeds\n2. Practice crop rotation\n3. Maintain proper plant spacing\n4. Water at the base of plants",
                "resources": ["https://extension.umn.edu/diseases/bacterial-leaf-spot"]
            },
            "fungus": {
                "treatment": "1. Apply fungicides containing chlorothalonil or mancozeb\n2. Remove severely infected plants\n3. Improve air circulation\n4. Apply neem oil as organic treatment",
                "prevention": "1. Water plants in the morning\n2. Use mulch to prevent soil splash\n3. Sterilize pruning tools\n4. Avoid overcrowding plants",
                "resources": ["https://extension.psu.edu/fungal-diseases-on-houseplants"]
            },
            "healthy": {
                "treatment": "No treatment needed. Your plant is healthy!",
                "prevention": "1. Continue current care routine\n2. Monitor regularly for pests/diseases\n3. Maintain proper watering and fertilization",
                "resources": []
            },
            "pest": {
                "treatment": "1. Identify specific pest first\n2. Use insecticidal soap or neem oil\n3. For severe cases, use appropriate pesticide\n4. Remove heavily infested leaves",
                "prevention": "1. Regularly inspect plants\n2. Encourage beneficial insects\n3. Use physical barriers like row covers\n4. Keep garden clean of debris",
                "resources": ["https://extension.umn.edu/yard-and-garden-insects"]
            },
            "virus": {
                "treatment": "1. Remove and destroy infected plants\n2. Control insect vectors\n3. Disinfect tools after use\n4. No chemical treatment available",
                "prevention": "1. Use virus-free planting material\n2. Control aphids and other vectors\n3. Avoid working with plants when wet\n4. Remove weed hosts",
                "resources": ["https://extension.umn.edu/diseases/viral-diseases"]
            }
        }
    
    def get_remedies(self, disease):
        # Get remedies for the detected disease (case insensitive)
        disease_lower = disease.lower()
        for key in self.remedies_db:
            if key in disease_lower:
                return self.remedies_db[key]
        
        # Default response if disease not found in database
        return {
            "treatment": "Specific treatment not available. Consult a local agricultural expert.",
            "prevention": "General prevention: Maintain plant health, monitor regularly, and practice good sanitation.",
            "resources": []
        }
    
    def open_web_resources(self):
        disease = self.result_vars["Disease Detected:"].get()
        disease_lower = disease.lower()
        
        resources = []
        for key in self.remedies_db:
            if key in disease_lower:
                resources = self.remedies_db[key].get("resources", [])
                break
        
        if resources:
            for url in resources:
                webbrowser.open_new_tab(url)
        else:
            messagebox.showinfo("Information", "No specific online resources available for this condition.")

if __name__ == "__main__":
    root = tk.Tk()
    app = LeafDoctorApp(root)
    root.mainloop()